<a href="https://colab.research.google.com/github/K9Santosh/FMML_LAB_23B21A4539/blob/main/FMML_MODULE_1%2C_LAB_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1A.
Yes, averaging the validation accuracy across multiple splits generally gives more consistent and reliable results. This is because using multiple splits, as in **cross-validation**, helps account for variability in the data and reduces the likelihood of the model overfitting or underfitting to a particular train-test split. Here's why:

### 1. **Reduces Variability**
   When you train and validate a model on a single train-test split, the results might depend on the specific data points in that split. Some splits might contain easier or harder examples, leading to skewed performance results. By using multiple splits (such as in k-fold cross-validation), you smooth out these irregularities by ensuring the model is tested on various portions of the dataset.

### 2. **Provides a More Robust Estimate**
   Averaging the validation accuracy over several splits gives a better estimate of the model's generalization performance on unseen data. This way, the reported performance is less influenced by any particular data split.

### 3. **Mitigates Overfitting Risk**
   Training and evaluating a model on a single split can lead to overfitting if the model performs well on that split but poorly on others. Cross-validation (and averaging the results) ensures the model is validated on diverse subsets, giving a more realistic sense of its ability to generalize.

### 4. **More Reliable Comparison Between Models**
   When comparing different models, using averaged performance metrics across multiple splits allows for a fairer and more reliable comparison, as the impact of any single split's idiosyncrasies is minimized.

Thus, averaging validation accuracy over multiple splits can provide a more consistent, stable, and trustworthy assessment of your model’s performance.

2A.
Yes, averaging validation accuracy across multiple splits (e.g., using cross-validation) typically gives a more accurate estimate of test accuracy compared to using a single train-test split. This is because cross-validation provides a better assessment of how well your model is likely to generalize to unseen data. Here's why:

### 1. **Accounts for Dataset Variability**
   In any single split, some data points may be easier or harder for the model to predict, and the test set might not represent the full diversity of the data. Cross-validation uses multiple train-test splits, ensuring that every data point is tested at least once, which reduces the risk that the performance on any single test set is misleading.

### 2. **Less Dependent on Random Split**
   A single test split could randomly be too easy or too difficult, giving you an overly optimistic or pessimistic estimate of test accuracy. Cross-validation reduces this randomness by averaging performance over multiple splits, resulting in a more reliable and accurate estimate of how the model will perform on new data.

### 3. **Generalization Performance**
   Cross-validation gives a more robust view of the model’s ability to generalize to new data. Since the model is trained and evaluated on different portions of the data, the averaged accuracy reflects a more realistic expectation of its performance on a true test set.

### 4. **Mitigates Overfitting**
   By testing on different subsets of the data and averaging the results, cross-validation helps reduce overfitting to specific test splits. This means the resulting accuracy estimate is less likely to be inflated due to overfitting to one specific set of data.

In short, averaging validation accuracy across multiple splits provides a more accurate and reliable estimate of the model's performance on unseen data, which closely approximates how the model will perform on the actual test set.

3A.
The number of iterations (or folds) in cross-validation affects the quality and stability of the estimate of test accuracy. In general, **increasing the number of iterations (folds) tends to provide a better estimate**, but this improvement comes with trade-offs. Here's how the number of iterations impacts the estimate:

### 1. **Accuracy of the Estimate**
   - **Higher Iterations Provide More Reliable Estimates**: As you increase the number of iterations (e.g., moving from 5-fold to 10-fold cross-validation), each data point is tested in more diverse training-test split combinations. This leads to a more stable and accurate estimate of the model’s performance since the model is tested on a wider variety of data subsets.
   - **Law of Large Numbers**: More iterations give you more samples of model performance (on different splits), which improves the consistency and reduces the variance of the accuracy estimate. This is analogous to the law of large numbers, where averaging over a larger number of samples yields a result closer to the true mean.

### 2. **Variance of the Estimate**
   - **Lower Variance with More Iterations**: When the number of iterations is higher, the model is exposed to more variability in the data, leading to lower variance in the performance estimate. The effect of any particular train-test split is diminished when more splits are averaged together, leading to a more consistent and less fluctuating result.
   - **Fewer Iterations Can Lead to High Variance**: With fewer iterations (e.g., 3-fold or even a single train-test split), the estimate is highly dependent on the specific subsets of data chosen for training and testing, which can result in higher variance and a less stable estimate of performance.

### 3. **Bias of the Estimate**
   - **Higher Iterations May Slightly Reduce Bias**: In cross-validation, bias refers to the error introduced by how well the cross-validation process estimates the true generalization performance of the model. Using more iterations (like 10-fold) slightly reduces bias because the model is trained on a larger portion of the data in each iteration. In leave-one-out cross-validation (LOOCV), for example, each model is trained on almost all the data, which can give the lowest bias.

   - **However, High Bias is Usually Not a Big Issue in Cross-Validation**: The bias reduction from increasing the number of iterations is often marginal unless you're using very small datasets where LOOCV might make a significant difference.

### 4. **Computational Cost**
   - **More Iterations Increase Computational Cost**: The primary trade-off for using more iterations is the increased computational time. For example, in 10-fold cross-validation, the model is trained and tested 10 times, which takes significantly longer than 5-fold cross-validation. If you increase the number of iterations, this can be computationally expensive, especially with large datasets or complex models.
   - **Diminishing Returns**: Beyond a certain point (e.g., moving from 10 to 20 folds), the benefits of additional iterations in terms of improved accuracy estimation may be marginal, but the computational costs continue to rise.

### Key Takeaways
- **More iterations generally give a better estimate** by reducing the variance and slightly lowering the bias of the performance estimate.
- **10-fold cross-validation** is a common choice because it balances accuracy and computational efficiency.
- **Too few iterations (e.g., 3 or 5 folds)** may give unstable estimates with high variance.
- **Too many iterations (e.g., LOOCV)** may not provide significant improvement in the estimate but can be computationally expensive.

Therefore, while more iterations tend to give a better estimate of test accuracy, the improvement diminishes after a certain point, and it's important to balance this with the computational cost.

4A.
Increasing the number of iterations in cross-validation can help when dealing with very small datasets, but it is not a perfect solution on its own. Here's how it can help and what the limitations are:

### How Increasing Iterations Helps with Small Datasets

1. **More Effective Use of Data**:
   - With a very small dataset, each data point is valuable, and splitting it into training and validation sets can leave you with very limited data for training. By increasing the number of iterations (e.g., using **k-fold cross-validation** or even **leave-one-out cross-validation (LOOCV)**), you maximize the amount of data used for training in each iteration.
   - In **LOOCV**, for example, each fold uses all but one data point for training, which ensures the model is trained on nearly the full dataset for each iteration. This helps make the most out of a small dataset.

2. **More Stable Performance Estimates**:
   - When the dataset is small, results from a single train-test split can vary widely due to the small sample size, making performance estimates unreliable. Increasing the number of iterations reduces the variance of these estimates by averaging the results across more splits, which can lead to a more consistent performance measure.
   
### Limitations of Increasing Iterations with Small Datasets

1. **Overfitting Risk Remains**:
   - With small datasets, increasing the number of iterations alone does not fully eliminate the risk of overfitting. Even though you're using most of the data in each iteration, the model may still overfit to the small training sets, especially if the model is complex. Overfitting occurs when the model memorizes the small training data rather than learning generalizable patterns.
   
2. **Bias in LOOCV and Small Datasets**:
   - In **LOOCV**, the model is trained on all but one data point, which can result in lower bias (since almost all the data is used for training), but this also makes LOOCV particularly prone to **high variance** in small datasets. Each fold in LOOCV is trained on nearly the same data, which can lead to noisy predictions when just a single point is left out.
   - As a result, while LOOCV might seem ideal for small datasets, it often produces highly variable estimates of test accuracy, especially when the model is prone to overfitting.

3. **Lack of Representation in Small Datasets**:
   - Even with more iterations, a small dataset may simply not represent the underlying data distribution well enough. If the dataset is not diverse or representative of the problem you're trying to solve, increasing the number of iterations in cross-validation won't make up for the lack of data.
   
4. **Computational Cost**:
   - For small datasets, **LOOCV** or using a very high number of iterations can be computationally inefficient. While this is less of a concern with small datasets, the trade-off between the computational cost and the marginal gain in performance estimates should still be considered.

### Potential Solutions for Small Datasets Beyond More Iterations

1. **Data Augmentation**:
   - For small datasets, augmenting the data (especially in fields like image classification) can artificially increase the amount of training data. This helps reduce overfitting and improves the model’s ability to generalize, even without needing more iterations in cross-validation.

2. **Simpler Models**:
   - Using a simpler model with fewer parameters can help prevent overfitting when working with small datasets. A simple model is less likely to memorize the data and can generalize better even with limited training data.

3. **Regularization**:
   - Adding regularization techniques like L2 regularization or dropout (in neural networks) can help prevent overfitting by discouraging the model from relying too heavily on any particular set of data points. This allows the model to learn more general patterns.

4. **Bootstrapping**:
   - Instead of cross-validation, **bootstrapping** is another resampling technique that can help with small datasets. It involves randomly sampling (with replacement) from the dataset to create many training sets, which can be used to estimate model performance.

### Conclusion

Increasing the number of iterations (such as in k-fold cross-validation or LOOCV) can help deal with small datasets by making better use of the available data and providing more stable performance estimates. However, it is not a complete solution. Overfitting remains a concern, and the dataset may still lack sufficient information to provide meaningful generalizations. Combining increased iterations with techniques like data augmentation, simpler models, regularization, or bootstrapping can better address the challenges posed by small datasets.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.